In [ ]:
import numpy as np
import xarray as xr
from modules.ADI import ADI
from modules.classes import Quantity2D, Analytic, Interpolate
from modules.integrator import forward_euler_final
from modules.tests import plot_mass_conservation, calculate_boundary_flux, integrate_concentration
from modules.functions import animate
import matplotlib.pyplot as plt
from modules.tests import test_gaussian
import xarray as xr

# Boundary Conditions Test

In [ ]:
test_int1, test_analytic1 = test_gaussian(ADI, BC='dirichlet')
diff1 = test_int1 - test_analytic1


In [ ]:
xr.plot.contourf(test_analytic1.isel(t=900), levels=20)
plt.suptitle("Analytic Heat Kernel")

In [ ]:
xr.plot.contourf(diff1.isel(t=900), levels=30)
plt.title('Dirichlet')

In [ ]:
absdiff1 = np.absolute(diff1)
absdiff1.median(dim=('x','y')).plot()

In [ ]:
absdiff1.mean(dim=('x','y')).plot()

In [ ]:
test_int2, test_analytic2 = test_gaussian(ADI, BC='neumann') # Time to check whether the neumann conditions are correct
diff2 = test_int2 - test_analytic2

In [ ]:
xr.plot.contourf(diff2.isel(t=900), levels=30, cmap='Reds')
plt.title('Neumann')

In [ ]:
absdiff2 = np.absolute(diff2)
absdiff2.median(dim=('x','y')).plot()

In [ ]:
absdiff2.mean(dim=('x','y')).plot()

In [ ]:
test_int3, test_analytic3 = test_gaussian(ADI, BC='open')
diff3 = test_int3 - test_analytic3

In [ ]:
xr.plot.contourf(diff3.isel(t=900), levels=30)
plt.title('Pseudo-Open')

In [ ]:
absdiff3 = np.absolute(diff3)
absdiff3.median(dim=('x','y')).plot()

In [ ]:
absdiff3.mean(dim=('x','y')).plot()

In [ ]:
# Overlapped plots
data1 = absdiff1.median(dim=('x','y'))
data2 = absdiff2.median(dim=('x','y'))
data3 = absdiff3.median(dim=('x','y'))
tcoords = data1.coords['t']
plt.plot(tcoords, data1, label = 'Dirichlet')
plt.plot(tcoords, data3, label = 'Pseudo-Open', linestyle='--')
plt.plot(tcoords, data2, label = 'Neumann')
plt.title("Gaussian kernel median error, N_grid=50, N_time=1000")
plt.ylabel("$\Delta$ concentration")
plt.xlabel("time")
plt.grid()
plt.legend()

In [ ]:
# Overlapped plots
data1 = absdiff1.mean(dim=('x','y'))
data2 = absdiff2.mean(dim=('x','y'))
data3 = absdiff3.mean(dim=('x','y'))
tcoords = data1.coords['t']
plt.plot(tcoords, data1, label = 'Dirichlet')
plt.plot(tcoords, data3, label = 'Pseudo-Open', linestyle='--')
plt.plot(tcoords, data2, label = 'Neumann')
plt.title("Gaussian kernel mean error, N_grid=50, N_time=1000")
plt.ylabel("$\Delta$ concentration")
plt.xlabel("time")
plt.grid()
plt.legend()

In [ ]:
normalizer = test_analytic1.max(dim=('x','y'))
# Overlapped plots
data1 = absdiff1.max(dim=('x','y'))/normalizer
data2 = absdiff2.max(dim=('x','y'))/normalizer
data3 = absdiff3.max(dim=('x','y'))/normalizer
tcoords = data1.coords['t']
plt.plot(tcoords, data1, label = 'Dirichlet')
plt.plot(tcoords, data3, label = 'Pseudo-Open', linestyle='--')
plt.plot(tcoords, data2, label = 'Neumann')
plt.title("Normalized max error, N_grid=50, N_time=1000")
plt.ylabel("concentration fraction")
plt.xlabel("time")
plt.grid()
plt.legend()

In [ ]:
xr.plot.contourf(test_analytic1.isel(t=600)/test_analytic1.isel(t=600).max(), levels=40, cmap='nipy_spectral')
plt.suptitle("Normalized Analytic Heat Kernel")

# FTCS vs. Crank-Nicholson ADI

In [ ]:
test_int4, test_analytic4 = test_gaussian(forward_euler_final)
diff4 = test_int4 - test_analytic4


In [ ]:
xr.plot.contourf(diff4.isel(t=900), levels=30)

In [ ]:
absdiff4 = np.absolute(diff4)
absdiff4.median(dim=('x','y')).plot()

In [ ]:
diff1.mean(dim=('x','y')).plot(label='Crank-Nicholson')
diff4.mean(dim=('x','y')).plot(label='FTCS')
plt.legend() #Bruh

In [ ]:
absdiff4.mean(dim=('x','y')).plot()

In [ ]:
diff5 = (test_int4 - test_int1)
contour = xr.plot.contourf(diff5.isel(t=900), levels=30)
plt.suptitle("FTCS - CN, Gaussian IC, N_grid=50, timestep=900")
contour.set(label='normalized conc. fraction')

In [ ]:
diff5.mean(dim=('x', 'y')).plot()

In [ ]:
absdiff5 = np.abs(diff5)
absdiff5.mean(dim=('x','y')).plot()

In [ ]:
# Define the test parameters
n_grid = 100
n_time = 20000
xrange = (-1, 1) # metres
trange = (0, 10) # t is on the order of thousands of years

# Based on this, dx = 0.4, dt = 0.5*0.4^2/0.1 = 0.008, choose dt = 0.005
conc = Quantity2D(
    n_grid,
    n_time,
    xrange,
    xrange,
    trange,
)

diffarr = 0.01*np.ones((10, 10)) # in m^2 y^-1
diffarr[:,4:-4] = 0.1 # Vary along x; partial_x should be constant
xint = np.linspace(-1, 1, 10)
yint = np.linspace(-1, 1, 10)
diffusion = Interpolate(diffarr, xint, yint, s=0)

diffusion.plot_2D(func='func')

In [ ]:
xcoords = conc.xcoords
ycoords = conc.ycoords
X, Y = np.meshgrid(xcoords, ycoords)
initial_condition = 1*np.exp(- (X**2 + Y**2)/(0.1)**2)
sources =  1*np.exp(- (X**2 + Y**2)/(0.1)**2)
#initial_condition = np.empty_like(X)
#initial_condition[n_grid//2, n_grid//2] = 100

fig = plt.figure()
ax = plt.axes(projection='3d')
ax.plot_surface(X, Y, sources)

In [ ]:
result_ds = forward_euler_final(conc, diffusion, initial_condition, sources)

In [ ]:
xr.plot.contourf(result_ds['concentration'].isel(t=500), levels=30)

In [ ]:
cumflux, totalconc, inflow, conserved_mass = plot_mass_conservation(result_ds)
plt.title("FTCS scheme, 5000 timesteps")
plt.ylabel("concentration")

In [ ]:
#result2_ds = ADI(conc, diffusion, initial_condition, sources, BC='dirichlet')
result2_ds = xr.open_dataset("result2_ds.nc")


In [ ]:
#result2_ds.to_netcdf("result2_ds.nc")

In [ ]:
diffds = result_ds - result2_ds

In [ ]:
xr.plot.contourf(diffds['concentration'].isel(t=800), levels=30)

In [ ]:
diffds.mean(dim=('x','y'))['concentration'].plot()
plt.title("(FTCS - CN) mean discrepancy")

In [ ]:
xr.plot.contourf(result2_ds['concentration'].isel(t=1000), levels=30)

In [ ]:
boundary = result_ds.isel(x=0).mean(dim='y')
boundary['concentration'].plot()

In [ ]:
cumflux2, totalconc2, inflow2, conserved_mass2 = plot_mass_conservation(result2_ds)
plt.title("Crank-Nicholson-ADI Scheme, 5000 timesteps")
plt.ylabel("concentration")

In [ ]:
plt.plot(result_ds.coords['t'], conserved_mass/conserved_mass[0], label='dx = 0.02')
plt.plot(result2_ds.coords['t'], conserved_mass2/conserved_mass2[0], label='dx = 0.04')
plt.title("Mass Conservation Normalized vs. step size")
plt.ylabel("Concentration Fraction")
plt.xlabel("timestep")
plt.legend()

In [ ]:
plot_mass_conservation(result_ds)
plot_mass_conservation(result2_ds)
plt.title("Crank-Nicholson-ADI Scheme, 5000 timesteps")
plt.ylabel("concentration")